## 0. 환경구성
### 0.1 패키지 Import


In [ ]:
# 기초 함수 Import
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### 0.2 구글 드라이브 연결


In [ ]:
!pip install kaggle

In [ ]:
ls -1ha kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 코드추가
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [ ]:
# 데이터 적제
item_categories = pd.read_csv("/gdrive/My Drive/kaggle1/item_categories.csv")
items=pd.read_csv('/gdrive/My Drive/kaggle1/items.csv')
shops=pd.read_csv('/gdrive/My Drive/kaggle1/shops.csv')
test=pd.read_csv('/gdrive/My Drive/kaggle1/test.csv')
sp=pd.read_csv('/gdrive/My Drive/kaggle1/sample_submission.csv')
trn=pd.read_csv('/gdrive/My Drive/kaggle1/sales_train_v2.csv')

## 1. 데이터 전처리

### 1-1. 데이터 효율화

In [ ]:
# 테스트에 있는 데이터만 선별
test_items_u = test.item_id.unique()
trn = trn[trn.item_id.isin(test_items_u)]

test_shops_u = shops.shop_id.unique()
trn = trn[trn.shop_id.isin(test_shops_u)]

### 1-2. 데이터 속성확인

In [ ]:
# 속성 별 데이터 확인
# 각 열을 for 문을 통해서 출력
for col in trn.columns:
    print('{}\n'.format(trn[col].head()))
# '{}\n' 줄 띠어 쓰기를 의미.  

In [ ]:
# 수치형 변수 확인하기

n_cols = [col for col in trn.columns[4:6]]
trn[n_cols].describe()

### 1-3. Input 형식 만들기

In [ ]:
test.head()
# Test의 값에 ID, Shop_ID, Item_ID 가 있음으로, 이를 기준으로 형식으로 만들어야함.

In [ ]:
# Month, Item ID, Shop ID 세개의 열을 이용해, 인풋 값을 만들자.

# 곱집합을 이용해, 빈 형식을 만들기
from itertools import product 

## Shop List와 Items List를 곱집합함
S_I_Prod = pd.DataFrame(list(product(test_shops_u, test_items_u)), columns=['shop_id','item_id']) 

# 모든 경우에 수에 알맞게, 트레인 값 병합
trn_B = pd.merge(S_I_Prod , trn, on=['shop_id','item_id'], how='left') 
trn_B['item_cnt_day'].fillna(0, inplace=True) 

# 그룹핑
trn_grp = trn_B.groupby(['shop_id','item_id'])

In [ ]:
trn_B

## 2. 파생변수 만들기

### 2.1 그룹 별 통계변수 추가

In [ ]:
# 그룹별 합계, 빈도, 평균, 표준편차

trn_IC = pd.DataFrame(trn_grp.agg({'item_cnt_day':['sum','count','mean','std'],'item_price':['sum', 'count','mean', 'std']})).reset_index() 
#trn_IC.columns = ['date_block_num','shop_id','item_id','item_cnt_sum','item_cnt_count','item_cnt_mean','item_cnt_std''item_price_sum','item_price_count','item_price_mean','item_price_std'] 

In [ ]:
trn_IC.columns = ['shop_id','item_id','item_cnt_sum','item_cnt_count','item_cnt_mean','item_cnt_std','item_price_sum','item_price_count','item_price_mean','item_price_std'] 

In [ ]:
trn_IC.head()

In [ ]:
trn_IC['item_cnt_count'].describe()

### 2.2 아이탬, 상점 그룹 추가

In [ ]:
pip install googletrans   #package 설치

In [ ]:
# 러시아어 번역을 위한 함수
from googletrans import Translator

trans = Translator()
def translate(col):
    en_list=[]
    for word in col:
        en = trans.translate(word, src = 'ru', dest='en')   #러시아어(ru) -> 영어(en)로 번역
        en_list.append(en.text)   #Tranlator().text = 번역결과
    return en_list

In [ ]:
# 10개만 선별
item_categories10 = item_categories.loc[1:10]
shop10 = shops.loc[1:10]

In [ ]:
item_categories10['item_category_name'] = translate(item_categories10['item_category_name'])
shop10['shop_name'] = translate(shop10['shop_name'])

In [ ]:
item_categories10
# 카테고리 이름의 첫 단어가 하위 항목을 포함하고 있음을 확인 가능

In [ ]:
shop10
# 상점 이름의 첫 단어가 지역명임을 확인 가능

In [ ]:
#카테고리 그룹 추가하기

# 카테고리의 처음이름 가져오기
item_grp = item_categories['item_category_name'].apply(lambda x: str(x).split(' ')[0]) 
# 코드 형식으로 바꿔 열 추가
item_categories['item_group'] = pd.Categorical(item_grp).codes 
# 아이탬에 그룹표기 추가
items = pd.merge(items, item_categories.loc[:,['item_category_id','item_group']], on=['item_category_id'], how='left') 
items.head()


In [ ]:
# 병합을 위한 테이블 생성
items_I = items[['item_id','item_category_id','item_group']]

In [ ]:
#Shop에서 도시 추가하기

city = shops.shop_name.apply(lambda x: str.replace(x, '!', '')).apply(lambda x: x.split(' ')[0]) 
shops['city'] = pd.Categorical(city).codes 
shops.head()

In [ ]:
# 병합을 위한 테이블 생성
shops_I = shops[['shop_id','city']]

### 2.3 파생 변수 합치기


In [ ]:
# 아이탬 카테고리 합치기
trn_ICI = pd.merge(trn_IC, items_I, on=['item_id'])

In [ ]:
# 상점 도시 합치기
trn_ICII = pd.merge(trn_IC, shops_I, on=['shop_id'])

In [ ]:
trn_ICII.head()

In [ ]:
# 같은 정보를 담는 테스트 데이터 생성
# 모든 경우에 수에 알맞게, 트레인 값 병합
test_B = pd.merge(S_I_Prod, test, on=['shop_id','item_id'], how='left') 


In [ ]:
test_B.head()

In [ ]:
len(test)

In [ ]:
len(test_B)

In [ ]:
#r결측치 제거
test_B=test_B.dropna(axis=0)
len(test_B)

In [ ]:
# 아이템 카테고리 합치기
test_ICI = pd.merge(test_B, items_I, on=['item_id'], how= 'left')
# 상점 도시 합치기
test_ICII = pd.merge(test_B, shops_I, on=['shop_id'],how= 'left')

In [ ]:
test_F = pd.merge(test_B, trn_IC, on=['shop_id','item_id'], how= 'left')
test_B['ID'].describe()

In [ ]:
test_F.head()

In [ ]:
del test_F['item_cnt_count']
#예측해야하는 값이므로 제거함.
test_F.head()

In [ ]:
# 결측치는 0으로 변경
test_F.fillna(0, inplace=True)
test_B['ID'].describe()

## 3. 데이터 모델링

In [ ]:
# 랜덤 포레스트를 이용
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# 트리개수 선정
rfc = RandomForestClassifier(n_estimators=40)
rfc

In [ ]:
# 학습을 위한 변수 설정

train = trn_ICII.copy()
train['item_cnt_count']

In [ ]:
train_Y = train['item_cnt_count']
train_Y.fillna(0, inplace=True)
train_Y.describe()

In [ ]:
train_X =trn_ICII.copy()
train_X.fillna(0, inplace=True)
train_X.head()
del train_X['item_cnt_count']
train_X.head()

In [ ]:
# 모델 구축
rfc.fit(train_X,train_Y)

In [ ]:
# 예측
 prediction=rfc.predict(test_F)

In [ ]:
test_F["item_cnt_month"] = prediction

In [ ]:
test_F.head()

In [ ]:
test_F["item_cnt_month"].describe()

In [ ]:
 Result = test_F[["ID","item_cnt_month"]]

In [ ]:
Result.head()

In [ ]:
Result.info()

In [ ]:
Result.ID = Result['ID'].astype('int')
Result.item_cnt_month = Result.item_cnt_month.astype('int')

In [ ]:
Result.to_csv('submission.csv',index=False)

In [ ]:
#제출
!kaggle competitions submit -c competitive-data-science-predict-future-sales -f submission.csv -m "Message"